In [1]:
import json
import requests

root_url = "http://localhost:8000/"

### Create a user

In [2]:
user1_data = {
	"@id": "ark:99999/test-user1",
	"name": "Test User1",
	"type": "Person",
	"email": "testuser1@example.org",
	"password": "test1",
	"organizations": [],
	"projects": [],
	"datasets": [],
	"software": [],
	"computations": [],
	"evidencegraphs": []
	}

# create user1
create_user1 = requests.post(root_url + "user", data=json.dumps(user1_data))
create_user1.json()

{'created': {'@id': 'ark:99999/test-user1',
  '@type': 'Person',
  'name': 'Test User1'}}

### Create a Software

In [3]:
# create software
software_data = {
	"@id": "ark:99999/test-software",
	"@type": "evi:Software",
	"name": "test-software",
	"owner": {
		"@id": "ark:99999/test-user1",
		"@type": "Person",
		"name": "Test User1",
		"email": "testuser1@example.org"
		}#,
	#"author": "some author(s)",
    #"downloadUrl": "some url",
    #"citation": "some citation",
	#"evi:usedBy": []
}

software_create = requests.post(root_url + f"software/", data=json.dumps(software_data))
software_create.json()

{'created': {'@id': 'ark:99999/test-software', '@type': 'evi:Software'}}

### Create a dataset

In [4]:
# create dataset
dataset_data = {
	"@id": "ark:99999/test-dataset",
	"@type": "evi:Dataset",
	"name": "test-dataset",
	"owner": {
		"@id": "ark:99999/test-user1",
		"@type": "Person",
		"name": "Test User1",
		"email": "testuser1@example.org"
		}
}

dataset_create = requests.post(root_url + f"dataset/", data=json.dumps(dataset_data))
dataset_create.json()

{'created': {'@id': 'ark:99999/test-dataset', '@type': 'evi:Dataset'}}

### Create metadata for the dataset to be uploaded

In [5]:
# create DataDownload metadata for a Dataset
test_data_download_for_dataset = {
	"@id": "ark:99999/test-download-dataset",
	"name": "random_numbers.txt",
	"@type": "DataDownload",
	"encodingFormat": ".txt",
	"encodesCreativeWork": dataset_data["@id"]
}

data_download_create = requests.post(root_url + f"datadownload", data=json.dumps(test_data_download_for_dataset))
data_download_create.json()

{'created': {'@id': 'ark:99999/test-download-dataset',
  '@type': 'DataDownload',
  'name': 'random_numbers.txt'}}

### Upload the dataset

In [6]:
# upload a dataset to minio object store
data_files = {'file': ('random_numbers.txt', '1\n2\n3\n4\n5')}
data_download_upload = requests.post(root_url + f"datadownload/ark:99999/test-download-dataset/upload", files=data_files)
data_download_upload.json()

{'updated': {'@id': 'ark:99999/test-download-dataset',
  '@type': 'DataDownload',
  'name': 'random_numbers.txt'}}

In [7]:
# read the DataDownload metadata for the dataset
data_download_read = requests.get(root_url + f"datadownload/ark:99999/test-download-dataset")
data_download_read.json()

'{"@id": "ark:99999/test-download-dataset", "@type": "DataDownload", "contentSize": "9", "contentUrl": "test-dataset/random_numbers.txt", "sha256": null, "uploadDate": "2022-07-01T13:20:35+00:00", "version": "1.0", "@context": {"@vocab": "https://schema.org/", "evi": "https://w3id/EVI#"}, "name": "random_numbers.txt", "encodingFormat": ".txt", "encodesCreativeWork": {"@id": "ark:99999/test-dataset", "@type": "evi:Dataset", "name": "test-dataset"}}'

### Create metadata for the script to be uploaded

In [8]:
# create DataDownload metadata for a Software
test_data_download_for_software = {
	"@id": "ark:99999/test-download-software",
	"name": "sum_script.py",
	"@type": "DataDownload",
	"encodingFormat": ".py",
	"encodesCreativeWork": software_data["@id"]
}

data_download_create = requests.post(root_url + f"datadownload", data=json.dumps(test_data_download_for_software))
data_download_create.json()

{'created': {'@id': 'ark:99999/test-download-software',
  '@type': 'DataDownload',
  'name': 'sum_script.py'}}

### Upload the script (Required: Create 'input-script' directory inside 'SOURCE_VOL' and store 'sum_script.py' for this operation)

In [9]:
# upload a software to minio object store
script_file = {'file': open('/home/sadnan/compute-test/input-script/sum_script.py', 'r')}
data_download_upload = requests.post(root_url + f"datadownload/ark:99999/test-download-software/upload", files=script_file)
data_download_upload.json()

{'updated': {'@id': 'ark:99999/test-download-software',
  '@type': 'DataDownload',
  'name': 'sum_script.py'}}

In [10]:
# read the DataDownload metadata for the script
data_download_read = requests.get(root_url + f"datadownload/ark:99999/test-download-software")
data_download_read.json()

'{"@id": "ark:99999/test-download-software", "@type": "DataDownload", "contentSize": "608", "contentUrl": "test-software/sum_script.py", "sha256": null, "uploadDate": "2022-07-01T13:20:59+00:00", "version": "1.0", "@context": {"@vocab": "https://schema.org/", "evi": "https://w3id/EVI#"}, "name": "sum_script.py", "encodingFormat": ".py", "encodesCreativeWork": {"@id": "ark:99999/test-software", "@type": "evi:Software", "name": "test-software"}}'

### Create a Computation

In [36]:
# create computation
computation_data = {
	"@id": "ark:99999/test-computation",
	"@type": "evi:Computation",
	"name": "test computation",
	"owner": {
		"@id": "ark:99999/test-user1",
		"@type": "Person",
		"name": "Test User1",
		"email": "testuser1@example.org"
		},
	# "author" : "",
    # "dateCreated" : "",
    # "dateFinished" : "",
    # "associatedWith" : [],
	"container": "python:alpine",
    "usedSoftware" : "ark:99999/test-software",
    "usedDataset" : "ark:99999/test-dataset",
	"containerId": ""
}

computation_create = requests.post(root_url + f"computation/", data=json.dumps(computation_data))
computation_create.json()

{'created': {'@id': 'ark:99999/test-computation', '@type': 'evi:Computation'}}

### Read computation

In [33]:
# read specific computation
get_computation = requests.get(root_url + f"computation/{computation_data['@id']}")
get_computation.json()

{'@id': 'ark:99999/test-computation',
 '@type': 'evi:Computation',
 'container': 'python:alpine',
 'containerId': '',
 '@context': {'@vocab': 'https://schema.org/', 'evi': 'https://w3id.org/EVI#'},
 'name': 'test computation',
 'owner': {'@id': 'ark:99999/test-user1',
  '@type': 'Person',
  'name': 'Test User1',
  'email': 'testuser1@example.org'},
 'usedSoftware': 'ark:99999/test-software',
 'usedDataset': 'ark:99999/test-dataset'}

# Perform computation by running the script on the dataset

In [37]:
# run the computation
compute_run = requests.put(root_url + f"computation/{computation_data['@id']}/execute")
compute_run.json()

{'message': 'launched container'}

In [26]:
get_computation = requests.get(root_url + f"computation/ark:99999/test-computation")
get_computation.json()

{'@id': 'ark:99999/test-computation',
 '@type': 'evi:Computation',
 'container': 'python:alpine',
 'containerId': 'ce6227c4f16fa4fa4d23338cb95ca7f37c5ba203d697de6735972d40fb7238ac',
 '@context': {'@vocab': 'https://schema.org/', 'evi': 'https://w3id.org/EVI#'},
 'name': 'test computation',
 'owner': {'@id': 'ark:99999/test-user1',
  '@type': 'Person',
  'name': 'Test User1',
  'email': 'testuser1@example.org'},
 'usedSoftware': 'ark:99999/test-software',
 'usedDataset': 'ark:99999/test-dataset',
 'dateCreated': '2022-07-01T18:55:33.9669797Z',
 'dateFinished': '2022-07-01T18:55:44.017969698Z'}